# Case 14 Powerflow Solver Test

## Run simulation

### Import Libraries

In [ ]:
import subprocess, sys, os
import urllib.request

dpsim_root_dir = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')
sys.path.insert(0, os.path.join(dpsim_root_dir, 'python/src/dpsim/'))
sys.path.insert(0, os.path.join(dpsim_root_dir, 'build'))

import matpower
import dpsimpy
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import urllib.request
import matplotlib.pyplot as plt

### Get simulation files

In [ ]:
if not os.path.exists('ieee14-data'):
    os.mkdir('ieee14-data')

url_static = 'https://github.com/martinmoraga/dpsim_data/raw/main/IEEE14/matpower/ieee14.mat'
url_dynamic = 'https://github.com/martinmoraga/dpsim_data/raw/main/IEEE14/matpower/ieee14_dyn.mat'
local_file_static = './ieee14-data/ieee14.mat'
local_file_dynamic = './ieee14-data/ieee14_dyn.mat'
urllib.request.urlretrieve(url_static, local_file_static)
urllib.request.urlretrieve(url_dynamic, local_file_dynamic)

### 1. Powerflow for initialization

In [ ]:
sim_name_pf = 'IEEE14_PF'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_pf)

# read and create dpsim topology
mpc_reader = matpower.Reader(mpc_file_path=local_file_static, mpc_name='ieee14')
mpc_reader.load_mpc(domain=matpower.Domain.PF)
system_pf = mpc_reader.system

# log results
logger = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
    logger.log_attribute(node.name()+'.S', 's', node)

# Parametrize and run simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.info)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(0.1)
sim_pf.set_final_time(0.1)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.add_logger(logger)
sim_pf.run()

### Read pf results

### 2. Dynamic simulation with fault at Node 14

#### SP - Dynamic Simulation

In [ ]:
sim_name_dyn_sp = 'SP_IEEE14_Dyn_InitPF'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_dyn_sp)

# read and create dpsim topology
mpc_reader = matpower.Reader(mpc_file_path=local_file_static, mpc_name='ieee14',
                             mpc_dyn_file_path=local_file_dynamic, mpc_dyn_name='ieee14_dyn')
mpc_reader.create_dpsim_objects(domain=matpower.Domain.SP, frequency=60, 
                                 with_avr=False, with_tg=False, with_pss=False)

### Extend topology with switch between node N15 and groud
sw = dpsimpy.sp.ph1.Switch('Fault', dpsimpy.LogLevel.info)
switch_closed = 0.1 * (13.8*13.8/100.0)
sw.set_parameters(1e18, switch_closed)
sw.open()
mpc_reader.dpsimpy_comp_dict['Fault'] = [sw]
mpc_reader.dpsimpy_comp_dict['Fault'].append([dpsimpy.sp.SimNode.gnd, mpc_reader.dpsimpy_busses_dict["N14"]])

# create dpsim topology
system_dyn = mpc_reader.create_dpsim_topology()

#initialize node voltages using pf results
system_dyn = mpc_reader.system
system_dyn.init_with_powerflow(system_pf, dpsimpy.Domain.SP)

# log results
logger = dpsimpy.Logger(sim_name_dyn_sp)
for node in system_dyn.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
     
# Parametrize and run simulation
sim = dpsimpy.Simulation(sim_name_dyn_sp, dpsimpy.LogLevel.info)
sim.set_system(system_dyn)
sim.set_time_step(1e-3)
sim.set_final_time(10)
sim.set_domain(dpsimpy.Domain.SP)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.KLU)
sim.do_init_from_nodes_and_terminals(True)
sim.do_system_matrix_recomputation(True)
sim.add_logger(logger)

# add event
sw_event_1 = dpsimpy.event.SwitchEvent(0.2, sw, True)
sw_event_2 = dpsimpy.event.SwitchEvent(0.3, sw, False)
sim.add_event(sw_event_1)
sim.add_event(sw_event_2)

sim.run()

#### DP - Dynamic Simulation

In [ ]:
sim_name_dyn_dp = 'DP_IEEE14_Dyn_InitPF'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_dyn_dp)

# read and create dpsim topology
mpc_reader = matpower.Reader(mpc_file_path=local_file_static, mpc_name='ieee14',
                             mpc_dyn_file_path=local_file_dynamic, mpc_dyn_name='ieee14_dyn')
mpc_reader.create_dpsim_objects(domain=matpower.Domain.DP, frequency=60, 
                                 with_avr=False, with_tg=False, with_pss=False)

### Extend topology with switch between node N15 and groud
sw = dpsimpy.dp.ph1.Switch('Fault', dpsimpy.LogLevel.info)
switch_closed = 0.1 * (13.8*13.8/100.0)
sw.set_parameters(1e18, switch_closed)
sw.open()
mpc_reader.dpsimpy_comp_dict['Fault'] = [sw]
mpc_reader.dpsimpy_comp_dict['Fault'].append([dpsimpy.sp.SimNode.gnd, mpc_reader.dpsimpy_busses_dict["N14"]])

# create dpsim topology
mpc_reader.create_dpsim_topology()

#initialize node voltages using pf results
system_dyn = mpc_reader.system
system_dyn.init_with_powerflow(system_pf, dpsimpy.Domain.DP)

# log results
logger = dpsimpy.Logger(sim_name_dyn_dp)
for node in system_dyn.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
     
# Parametrize and run simulation
sim = dpsimpy.Simulation(sim_name_dyn_dp, dpsimpy.LogLevel.info)
sim.set_system(system_dyn)
sim.set_time_step(1e-3)
sim.set_final_time(10)
sim.set_domain(dpsimpy.Domain.SP)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.KLU)
sim.do_init_from_nodes_and_terminals(True)
sim.do_system_matrix_recomputation(True)
sim.add_logger(logger)

# add event
sw_event_1 = dpsimpy.event.SwitchEvent(0.2, sw, True)
sw_event_2 = dpsimpy.event.SwitchEvent(0.3, sw, False)
sim.add_event(sw_event_1)
sim.add_event(sw_event_2)

sim.run()

#### EMT - Dynamic Simulation

In [ ]:
sim_name_dyn_emt = 'EMT_IEEE14_Dyn_InitPF'
dpsimpy.Logger.set_log_dir('logs/' + sim_name_dyn_emt)

# read and create dpsim topology
mpc_reader = matpower.Reader(mpc_file_path=local_file_static, mpc_name='ieee14',
                             mpc_dyn_file_path=local_file_dynamic, mpc_dyn_name='ieee14_dyn')
mpc_reader.create_dpsim_objects(domain=matpower.Domain.EMT, frequency=60, log_level=dpsimpy.LogLevel.info, 
                                 with_avr=False, with_tg=False, with_pss=False)

### Extend topology with switch between node N15 and groud
sw = dpsimpy.emt.ph3.SeriesSwitch('Fault', dpsimpy.LogLevel.info)
switch_closed = 0.1 * (13.8*13.8/100.0)
sw.set_parameters(1e18, switch_closed)
sw.open()
mpc_reader.dpsimpy_comp_dict['Fault'] = [sw]
mpc_reader.dpsimpy_comp_dict['Fault'].append([dpsimpy.emt.SimNode.gnd, mpc_reader.dpsimpy_busses_dict["N14"]])

# create dpsim topology
mpc_reader.create_dpsim_topology()

#initialize node voltages using pf results
system_dyn = mpc_reader.system
system_dyn.init_with_powerflow(system_pf, dpsimpy.Domain.EMT)

# log results
logger = dpsimpy.Logger(sim_name_dyn_emt)
for node in system_dyn.nodes:
    logger.log_attribute(node.name()+'.V', 'v', node)
     
# Parametrize and run simulation
sim = dpsimpy.Simulation(sim_name_dyn_emt, dpsimpy.LogLevel.debug)
sim.set_system(system_dyn)
sim.set_time_step(1e-4)
sim.set_final_time(10)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_solver(dpsimpy.Solver.MNA)
sim.set_direct_solver_implementation(dpsimpy.DirectLinearSolverImpl.KLU)
sim.do_init_from_nodes_and_terminals(True)
sim.do_system_matrix_recomputation(True)
sim.add_logger(logger)

# add events
sw_event_1 = dpsimpy.event.SwitchEvent(0.2, sw, True)
sw_event_2 = dpsimpy.event.SwitchEvent(0.3, sw, False)
sim.add_event(sw_event_1)
sim.add_event(sw_event_2)

sim.set_time_step(1e-4)
sim.run()

#### Read results

In [ ]:
from villas.dataprocessing.timeseries import TimeSeries as ts
import villas.dataprocessing.plottools as pt

dpsim_result_file = 'logs/' + sim_name_dyn_sp + '/' + sim_name_dyn_sp + '.csv'
ts_dpsim_sp = read_timeseries_csv(dpsim_result_file)
phasors = ts.phasors(ts_dpsim_sp)

dpsim_result_file = 'logs/' + sim_name_dyn_dp + '/' + sim_name_dyn_dp + '.csv'
ts_dpsim_dp = read_timeseries_csv(dpsim_result_file)

dpsim_result_file = 'logs/' + sim_name_dyn_emt + '/' + sim_name_dyn_emt + '.csv'
ts_dpsim_emt = read_timeseries_csv(dpsim_result_file)

In [ ]:
timestep_common = 1e-4
t_begin = 0.0
t_end = 3
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)
time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)

#plot parameters
width = 12
height = 4

varname_dpsim = 'N14.V_0'
nominal_voltage = 69000

#convert dpsim voltage to magnitude value and per-unit for comparison with psat
dpsim_dp_values = (ts_dpsim_emt[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx]/nominal_voltage).real
    
plt.figure(figsize=(width, height))
plt.plot(time, dpsim_dp_values, label='EMT - DPsim')

plt.legend(loc='lower right')
plt.xlabel('time (s)')
plt.grid()
#plt.ylim([0.99, 1.06])
plt.xlim([0.15, 0.35])
plt.show()

#### Plot results

##### HV side

In [ ]:
pt.plot_timeseries(1, phasors['N1.V']['abs'])
pt.plot_timeseries(1, phasors['N2.V']['abs'])
pt.plot_timeseries(1, phasors['N3.V']['abs'])
pt.plot_timeseries(1, phasors['N4.V']['abs'])
pt.plot_timeseries(1, phasors['N5.V']['abs'])

#### LV side

In [ ]:
pt.plot_timeseries(1, phasors['N6.V']['abs'])
pt.plot_timeseries(1, phasors['N7.V']['abs'])
pt.plot_timeseries(1, phasors['N8.V']['abs'])
pt.plot_timeseries(1, phasors['N9.V']['abs'])
pt.plot_timeseries(1, phasors['N10.V']['abs'])
pt.plot_timeseries(1, phasors['N11.V']['abs'])
pt.plot_timeseries(1, phasors['N12.V']['abs'])
pt.plot_timeseries(1, phasors['N13.V']['abs'])
pt.plot_timeseries(1, phasors['N14.V']['abs'])

#### Validation against PSAT

#### Load PSAT results

In [ ]:
if not os.path.exists('ieee14-data'):
    os.mkdir('ieee14-data')

url = 'https://github.com/martinmoraga/dpsim_data/raw/main/IEEE14/PSAT/PSAT_IEEE14_Fault_N14_100uS_without_Controls_SG_4Order.txt'
local_file = './ieee14-data/PSAT_IEEE14_Fault_N14_100uS_without_Controls_SG_4Order.txt'
urllib.request.urlretrieve(url, local_file)
ts_psat = read_timeseries_dpsim(local_file)


### Plot results

In [ ]:
timestep_common = 1e-3
t_begin = 0.0
t_end = 3
begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)
time = np.linspace(t_begin, t_end, num=end_idx-begin_idx)

#plot parameters
width = 12
height = 4

def plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, ylabels, timestep_common=0.001):
   
    #convert dpsim voltage to magnitude value and per-unit for comparison with psat
    dpsim_dp_values = (ts_dpsim_dp[varname_dpsim].interpolate(timestep_common).values[begin_idx:end_idx]/nominal_voltage).real
    dpsim_sp_values_abs_pu = ts_dpsim_sp[varname_dpsim].interpolate(timestep_common).abs().values[begin_idx:end_idx]/nominal_voltage
    dpsim_dp_values_abs_pu = ts_dpsim_dp[varname_dpsim].interpolate(timestep_common).abs().values[begin_idx:end_idx]/nominal_voltage
    psat_values = ts_psat[varname_psat].interpolate(timestep_common).values[begin_idx:end_idx]
    
    plt.figure(figsize=(width, height))
    plt.plot(time, dpsim_dp_values_abs_pu, label='DP - DPsim')
    plt.plot(time, dpsim_sp_values_abs_pu, label='SP - DPsim')
    plt.plot(time, psat_values, '--', label='PSAT')

    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.grid()
    #plt.ylim([0.99, 1.06])
    #plt.xlim([8, 10])
    plt.show()
    
    plt.figure(figsize=(width, height))
    plt.plot(time, dpsim_dp_values_abs_pu, label='DP - DPsim')
    plt.plot(time, dpsim_sp_values_abs_pu, label='SP - DPsim')
    #plt.plot(time, dpsim_dp_values, '--', label='DP')
    plt.legend(loc='lower right')
    plt.xlabel('time (s)')
    plt.grid()
    plt.xlim([0.15, 0.4])
    plt.show()
      
    #calculate RMSE
    rmse = np.sqrt(((dpsim_sp_values_abs_pu - psat_values) ** 2).mean())
    print('RMSE {:s}  = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(varname_dpsim, rmse, rmse/1.0*100, 1.0))

#### HV side

In [ ]:
varname_dpsim = 'N1.V'
varname_psat = 'V_Bus01'
nominal_voltage = 69000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, "Voltage magnitude N1", timestep_common=0.001)

In [ ]:
varname_dpsim = 'N2.V'
varname_psat = 'V_Bus02'
nominal_voltage = 69000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, "Voltage magnitude N2", timestep_common=0.001)

In [ ]:
varname_dpsim = 'N3.V'
varname_psat = 'V_Bus03'
nominal_voltage = 69000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, "Voltage magnitude N3", timestep_common=0.001)

In [ ]:
varname_dpsim = 'N4.V'
varname_psat = 'V_Bus04'
nominal_voltage = 69000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, "Voltage magnitude N4", timestep_common=0.001)

In [ ]:
varname_dpsim = 'N5.V'
varname_psat = 'V_Bus05'
nominal_voltage = 69000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, "Voltage magnitude N5", timestep_common=0.001)

#### Low voltage side

In [ ]:
varname_dpsim = 'N6.V'
varname_psat = 'V_Bus06'
nominal_voltage = 13800
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, "Voltage magnitude N6", timestep_common=0.001)

In [ ]:
varname_dpsim = 'N7.V'
varname_psat = 'V_Bus07'
nominal_voltage = 13800
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, "Voltage magnitude N7", timestep_common=0.001)

In [ ]:
varname_dpsim = 'N8.V'
varname_psat = 'V_Bus08'
nominal_voltage = 18000
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, "Voltage magnitude N8", timestep_common=0.001)

In [ ]:
varname_dpsim = 'N9.V'
varname_psat = 'V_Bus09'
nominal_voltage = 13800
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, "Voltage magnitude N9", timestep_common=0.001)

In [ ]:
varname_dpsim = 'N10.V'
varname_psat = 'V_Bus10'
nominal_voltage = 13800
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, "Voltage magnitude N10", timestep_common=0.001)

In [ ]:
varname_dpsim = 'N11.V'
varname_psat = 'V_Bus11'
nominal_voltage = 13800
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, "Voltage magnitude N11", timestep_common=0.001)

In [ ]:
varname_dpsim = 'N12.V'
varname_psat = 'V_Bus12'
nominal_voltage = 13800
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, "Voltage magnitude N12", timestep_common=0.001)

In [ ]:
varname_dpsim = 'N13.V'
varname_psat = 'V_Bus13'
nominal_voltage = 13800
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, "Voltage magnitude N13", timestep_common=0.001)

In [ ]:
varname_dpsim = 'N14.V'
varname_psat = 'V_Bus14'
nominal_voltage = 13800
plot_node_volt_abs(varname_dpsim, varname_psat, ts_dpsim_sp, ts_dpsim_dp, ts_psat, nominal_voltage, "Voltage magnitude N14", timestep_common=0.001)